In [4]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# 파일 경로 설정
EMB_PATH = "train_embeddings.npy"
LABEL_PATH = "train_labels.npy"

# 1) 데이터 로드
embeddings = np.load(EMB_PATH)
labels = np.load(LABEL_PATH)

# 2) 기본 정보 출력
print(f"Embeddings shape: {embeddings.shape}")
print(f"Labels shape:     {labels.shape}")

# 3) 클래스 분포 확인
unique, counts = np.unique(labels, return_counts=True)
dist_df = pd.DataFrame({"class": unique, "count": counts})

# 4) 임베딩 벡터 노름(길이) 통계
norms = np.linalg.norm(embeddings, axis=1)
print(f"Embedding norms → mean: {norms.mean():.4f}, std: {norms.std():.4f}")

Embeddings shape: (436, 10, 256)
Labels shape:     (436,)
Embedding norms → mean: 0.9745, std: 0.8130


In [ ]:
import numpy as np
import torch
from mmengine.config import Config
from mmengine.runner import load_checkpoint
from mmaction.registry import MODELS
from sklearn.metrics import accuracy_score

# 파일 경로
EMB_PATH   = "train_embeddings.npy"
LABEL_PATH = "train_labels.npy"
CFG_PATH   = r"D:\mmaction2\configs\skeleton\stgcnpp\my_stgcnpp.py"
CKPT_PATH  = r"D:\mmaction2\checkpoints\stgcnpp_8xb16-joint-u100-80e_ntu60-xsub-keypoint-2d_20221228-86e1e77a.pth"

# 1) 임베딩·라벨 로드
embeddings = np.load(EMB_PATH)   # (N,10,256) or (N,256)
labels     = np.load(LABEL_PATH) # (N,)

# 비디오 단위 평균
if embeddings.ndim == 3:
    embeddings = embeddings.mean(axis=1)  # (N,256)

# 2) 체크포인트 불러와서 orig_num_classes 추출
ckpt = torch.load(CKPT_PATH, map_location="cpu")
state_dict = ckpt.get("state_dict", ckpt)
orig_num_classes = state_dict["cls_head.fc.weight"].shape[0]
print(f"원본 num_classes = {orig_num_classes}")

# 3) config 로드 후 cls_head.num_classes 덮어쓰기
cfg = Config.fromfile(CFG_PATH)
cfg.model.cls_head.num_classes = orig_num_classes

# 4) 모델 빌드 및 가중치 로드
model = MODELS.build(cfg.model).to("cpu")
load_checkpoint(model, CKPT_PATH, map_location="cpu", strict=False)

# 5) fc 레이어 가져오기
fc = model.cls_head.fc  # nn.Linear(256 -> orig_num_classes)

# 6) 임베딩 → logits 재생성 & 예측
with torch.no_grad():
    emb_tensor = torch.from_numpy(embeddings).float()       # (N,256)
    logits     = emb_tensor @ fc.weight.t() + fc.bias.unsqueeze(0)  # (N,orig_num_classes)
    preds      = logits.argmax(dim=1).numpy()               # (N,)

# 7) 정확도 계산
acc = accuracy_score(labels, preds)
print(f"🔍 Reconstructed head accuracy: {acc*100:.2f}%")

# 8) 샘플별 비교 (최대 5개)
for i in range(min(5, len(labels))):
    print(f"Sample {i}: GT={labels[i]}, Pred={preds[i]}")


Loads checkpoint by local backend from path: D:\mmaction2\checkpoints\stgcnpp_8xb16-joint-u100-80e_ntu60-xsub-keypoint-2d_20221228-86e1e77a.pth
🔍 Reconstructed head accuracy: 0.00%
Sample 0: GT=1, Pred=11, Logits=[1.0068751573562622, -1.1954060792922974, 2.093900442123413, 1.2456393241882324, -0.055919770151376724, -0.47261103987693787, -0.9784227609634399, -0.9783732891082764, -0.1777428835630417, 0.8454040288925171, 2.468869686126709, 4.358231544494629, -0.21862749755382538, -0.36530572175979614, -2.043872356414795, -0.12745419144630432, -0.12548817694187164, -0.6058102250099182, -1.741723656654358, -0.750993013381958, -1.5567537546157837, -0.041848957538604736, -1.2415812015533447, -0.4873270094394684, -0.5188603401184082, -0.29287073016166687, -0.46739399433135986, 0.1978524923324585, 4.170835494995117, 2.5379908084869385, -0.8198908567428589, 3.29093861579895, 0.22386717796325684, -0.9392722249031067, 0.6287589073181152, 1.2975068092346191, 0.014412004500627518, -0.012317758053541